# Tutorial: Maximum Cut via Quantum Imaginary Time Evolution (QITE)
## By: Willie Aboumrad, Senior Quantum Applications Scientist at IonQ

Quantum computers are poised to solve problems that are currently intractable,
and IonQ is leading the way. In this tutorial, we demonstrate how to leverage
[IonQ Forte][1]'s industry-leading capabilities to solve instances of the
[NP-hard][2] combinatorial optimization problem known as
[Maximum Cut (MaxCut)][3] using a
[novel Variational Quantum Imaginary Time Evolution (varQITE)][4] algorithm
developed by IonQ in conjunction with researchers at Oak Ridge National Labs
(ORNL).

[1]: https://ionq.com/forte
[2]: https://en.wikipedia.org/wiki/NP-hardness
[3]: https://en.wikipedia.org/wiki/Maximum_cut
[4]: https://arxiv.org/pdf/2404.16135

## What's the problem? 
### MaxCut 101

The Maximum Cut Problem (MaxCut) is a classic combinatorial optimization
problem commonly used as an algorithm benchmark by scientific computing
researchers. MaxCut is a graph problem: given a graph $G = (V, E)$ with
vertex set $V$ and edge set $E$, it asks for a partition of $V$ into sets
$S$ and $T$ maximizing the number of edges crossing between $S$ and $T$, 
🤔 **[or in more complex problem case - sum of weights of these crossed edges.]**

Formally, we write MaxCut as a [Quadratic Program (QP)][5] with binary
decision variables as follows. For each node $v \in V$, we let $x_v$ denote
a binary variable indicating whether $v$ belongs to $S$ or $T$. The objective
is to maximize the number of cut edges:
$$
    \text{maximize}_x\quad \frac{1}{2} \sum_{(v, w) \in E} x_v (1 - x_w).
$$

Without loss of generality, we can label the vertices $v \in V$ using
$1, 2, \ldots, n$ and re-express this QP in standard form using the
[Laplacian matrix][6] $L(G)$, which is the difference between the [degree][7]
and [adjacency][8] matrices. In symbols, MaxCut is the following QP:

$$
\begin{align}
    \text{minimize}& \quad \frac{1}{2} x^T L(G) x \\
    \text{subject to}& \quad x \in \{0, 1\}^n.
\end{align}
$$

[5]: https://en.wikipedia.org/wiki/Quadratic_programming
[6]: https://en.wikipedia.org/wiki/Laplacian_matrix
[7]: https://en.wikipedia.org/wiki/Degree_matrix
[8]: https://en.wikipedia.org/wiki/Adjacency_matrix

## As a Hamiltonian energy minimization problem
### Quantum MaxCut

Quantum computers are good at finding the ground state of particle systems
evolving under the action of a given Hamiltonian. In this section, we'll
use the [Ising map][9] **BROKEN LINK** to define a Hamiltonian whose energies are exactly
the values of the MaxCut objective function. This correspondence will
effectively translate our classical combinatorial optimization problem into
a quantum problem, which we'll approach using our novel heuristic.

Given an objective function $C(x)$, with domain $x \in \{0, 1\}^n$, the
Ising map produces a Hamiltonian $H_C$ on $n$ qubits such that
$$
    H_C \ket{x} = C(x) \ket{x}.
$$
In the last equation, $\ket{x}$ denotes the $n$-qubit
[computational basis][10] state indexed by the bit-string $x \in \{0, 1\}$.
Thus the last equation says each of the $2^n$ computational basis states is an
eigenvector of $H_C$, and the eigenvalue corresponding to $\ket{x}$ is $C(x)$;
that is, $H_C$ is diagonal with respect to the computational basis, and its
energies are the values of the objective function $C$.

The Ising map constructs the $H_C$ by replacing each $x_j$ in the formal
expression of $C(x)$ by the operator
$$
    \hat{X}_j \coloneqq \frac{1}{2}(I - Z_j),
$$
where $I$ denotes the identity operator on $n$ qubits and $Z_j$ denotes the
[Pauli-Z][11] operator acting on the $j$th qubit. Notice that $\hat{X}_j$ is
diagonal with respect to the computational basis, and its eigenvalues are zero
and one; in particular,
$$
    \hat{X}_j \ket{x} = x_j \ket{x}.
$$

### MaxCut Hamiltonian

When we apply the Ising map construction to the MaxCut objective
$$
    M(x) = \frac{1}{2} x^T L(G) x = \frac{1}{2} \sum_{j, k} \ell_{jk} \, x_j x_k,
$$
we obtain the Hamiltonian
$$
    H_M = \frac{1}{8} \sum_{j, k} \ell_{jk} (I - Z_j)(I - Z_k) = -\frac{1}{4} \sum_{j, k} \ell_{jk} Z_j Z_k.
$$

The code cell below illustrates the construction of the MaxCut Hamiltonian, via
the Ising map, corresponding to a simple graph.

**Note: no "Ising map" page on wikipedia**

[9]: https://en.wikipedia.org/wiki/Ising_map
[10]: https://en.wikipedia.org/wiki/Qubit#Standard_representation
[11]: https://en.wikipedia.org/wiki/Pauli_matrices

In [ ]:
import networkx as nx
from qiskit.quantum_info import SparsePauliOp

# Construct your favorite graph using NetworkX
edges = [(0, 1), (0, 2), (0, 3), (1, 2), (2, 3)]
graph = nx.Graph()
graph.add_edges_from(edges)
nx.draw(graph)

# Compute the MaxCut Hamiltonian
laplacian = nx.laplacian_matrix(graph).toarray()
num_nodes = graph.number_of_nodes()
quad_terms = list()
for j in range(num_nodes):
    for k in range(j + 1, num_nodes):
        if laplacian[j][k]:
            quad_terms.append(("ZZ", [j, k], laplacian[j][k]))
maxcut_ham = -1/2 * SparsePauliOp.from_sparse_list(quad_terms, num_qubits=num_nodes)
maxcut_ham -= 1/4 * SparsePauliOp("I" * num_nodes, laplacian.diagonal().sum())
print("Maxcut Hamiltonian:", maxcut_ham)

## Energy minimization via QITE
### Enter varQITE

With the MaxCut Hamiltonian in hand, we can turn to minimizing its energy using
our [novel quantum-classical varQITE heuristic][1]. Much like any Variataional
Quantum Algorithm (VQA), our novel varQITE method provides a recipe for
iteratively updating the parameters in a variational quantum circuit to
minimize the expectation value of the MaxCut Hamiltonian, measured with respect
to the parametrized state.

A key novelty is that our varQITE algorithm does *not* rely on a classical
optimizer to update the circuit parameters; instead, it specifies an explicit
update rule based on the solution of a system linear Ordinary Differential
Equations (ODEs). The ODEs relate the gradient of the variational circuit
parameters to the expected value of certain operators related to the MaxCut
Hamiltonian, and they are derived from an [Ehrenfest Theorem][3] that applies
to [imaginary time evolution][4]. For details, see Equation (5) in our [varQITE
paper][1].

In any case, setting up the ODE system at each step of the algorithm requires
executing a batch of quantum circuits and running some post-processing to
evaluate the results.

The code cell below illustrates how to set up the variational ansatz
$\ket{\Psi(\theta)}$ introduced by our [varQITE paper][1] in Equation (2).
We'll set up the required circuits and the ODEs further down.

[1]: https://arxiv.org/abs/2404.16135
[2]: https://www.nature.com/articles/s42254-021-00348-9
[3]: https://en.wikipedia.org/wiki/Ehrenfest_theorem
[4]: https://en.wikipedia.org/wiki/Imaginary_time

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

# Construct your favorite ansatz
ansatz = QuantumCircuit(graph.number_of_nodes())

# Use a uniform superposition as the initial state
ansatz.h(range(ansatz.num_qubits))
ansatz.barrier()

# Place entangling gates across qubit pairs that are connected in the graph
complete_graph = nx.complete_graph(graph.number_of_nodes())
params = ParameterVector(r"$\theta$", complete_graph.number_of_edges())
for theta, (u, v) in zip(params, complete_graph.edges):
    ansatz.cx(u, v)
    ansatz.ry(theta, v)
    ansatz.cx(u, v)
ansatz.draw("mpl")

We'll now use our ``ansatz`` to generate all the circuits that are needed to
compute the coefficients in the ODEs in Equation (5). We'll need $2n + 1$
circuits in total: $1$ to compute the expected value of $H_M$, and $2n$ to
compute its gradient with respect to the $n$ circuit parameters.

In [ ]:
import math

# Set the current iteration parameters
# Play with this value! Make it whatever you'd like!!!
curr_params = [0] * ansatz.num_parameters

# Get the energy evaluation circuit
iter_circuits = list()
iter_circuits.append(ansatz.assign_parameters(curr_params))

# Get the 2n gradient circuits via the parameter shift rule
theta = ansatz.parameters
for k in range(ansatz.num_parameters):
    for pm in range(2):
        pm_shift = {symb: val + (-1)**pm * (j == k) * math.pi / 2 for j, (symb, val) in enumerate(zip(theta, curr_params))}
        iter_circuits.append(ansatz.assign_parameters(pm_shift))

# Sanity check
assert len(iter_circuits) == 2 * ansatz.num_parameters + 1, "Incorrect number of circuits!"

We can now execute the required ``iter_circuits`` using an IonQ QPU like IonQ
Forte! 

The code cell below illustrates how to run this calculation. Replace the
``"simulator"`` target by ``"forte-1"`` when you're ready to run on the real
QPU!

Make sure you've exported your IonQ API key as the ``IONQ_API_KEY`` environment
variable before running the next cell!

In [ ]:
#from qiskit_ionq import Backend # was: from ionq import Backend
from qiskit_ionq import IonQProvider

provider = IonQProvider("WQxiqW9mttN6lZLJqkOdiEHdiZ9dgUrS")
backend = provider.get_backend("ionq_simulator")

job = backend.run(iter_circuits, shots=1000, name="QITE iteration")

# Get the results
counts = job.get_counts()

With the circuit counts in hand, we can run some post-processing classical code
to set up and solve the ODEs described in Equation (5). 

In this demo, we've used optimized parameters from a previous run. So we can
retrieve varQITE's top solution candidates from the counts of the zeroth circuit.

In [ ]:
from qiskit_optimization.applications import Maxcut

top_candidates = sorted(counts, key=lambda bitstring: counts[bitstring], reverse=True)

# Draw the best candidate
Maxcut(graph).draw(top_candidates[0])

In [ ]:
!pip install qiskit_optimization